In [36]:
period_length_sec = 3
fps = 30
import bar_chart_race as bcr
import matplotlib.pyplot as plt
# Set up a figure looking like bar_chart_race's default.
# fig, ax = plt.subplots(figsize=(4, 2.5), dpi=144)
# ax.set_facecolor('.8')
# ax.tick_params(labelsize=8, length=0)
# ax.grid(True, axis='x', color='white')
# ax.set_axisbelow(True)
# [spine.set_visible(False) for spine in ax.spines.values()]
# Setting background to a solid color fixes the problem.
# fig.patch.set_facecolor('white')
def race(df, filename='EmissiesNL', pause=0):
    bcr.bar_chart_race(
        df=df,
        filename=filename+'.mp4',
        orientation='h',
        sort='desc',
        n_bars=18,
        fixed_order=False,
        fixed_max=False,
        steps_per_period=fps * period_length_sec,
        interpolate_period=False,
        end_period_pause=pause,
        period_label={'x': .99, 'y': .25, 'ha': 'right', 'va': 'center', 'size': 8},
        # period_fmt='%B %d, %Y',
        period_summary_func=lambda v, r: {'x': .99, 'y': .18,
                                        's': f'Top 10: {v.nlargest(10).sum():.0f} kton CO2',
                                        'ha': 'right', 'size': 8},
        period_template='Jaar: {x:.0f}', 
        
        period_length=period_length_sec * 1000,
        
        perpendicular_bar_func=None, 
        colors='dark12', 
        title={
            'label': f'Per {filename}: CO2 jaarlijkse uitstoot (kton)',
            'size': 8,
            'color': 'black',
            'loc': 'left',
            'pad': 8
        },
        
        
        bar_size=.95, 
        bar_textposition='inside',
        bar_texttemplate='{x:.0f}', 
        bar_label_font=7, 
        tick_label_font=7, 
        tick_template='{x:,.0f}',
        shared_fontdict=None, 
        scale='linear', 
        fig=None, 
        writer=None, 
        bar_kwargs={'alpha': .7},
        fig_kwargs={'figsize': (6, 3.5), 'dpi': 144},
        filter_column_colors=True
    )

In [37]:
import pandas as pd
import numpy as np
# https://pypi.org/project/bar-chart-race/
# pip install bar_chart_race
# pip install git+https://github.com/dexplo/bar_chart_race
# pip install odfpy

# https://www.cbs.nl/nl-nl/dossier/dossier-broeikasgassen/hoofdcategorieen/hoe-groot-is-onze-broeikasgasuitstoot-wat-is-het-doel-
# https://www.emissieautoriteit.nl/onderwerpen/rapportages-en-cijfers-eu-ets/documenten/publicatie/2021/05/31/factsheet-ets-uitstoot-2020
# https://www.emissieautoriteit.nl/documenten/publicatie/2021/05/05/ets-uitstoot-2020

def map_columns(df, newcol):
    emissie_mapping = pd.read_csv('emissie_mapping.csv', sep=';')
    mapping = {k:v for k,v in zip(emissie_mapping['From'], emissie_mapping[newcol])}
    return df.rename(columns=mapping)

file = 'Emissiecijfers+2013-2020.ods'

emissions = pd.read_excel(file, engine='odf')
emissions = emissions.T
emissions.columns = emissions.loc['Inrichtingsnaam',:]
emissions.fillna(0, inplace=True)
emissions.drop(0, axis=1,inplace=True)

In [38]:
emissions.replace('nan', 0, inplace=True)
emissions.replace('NaN', 0, inplace=True)
emissions.replace('-', 0, inplace=True)
filtered_emissions = emissions[4:]
cols = filtered_emissions.columns
for c in cols:
    filtered_emissions[c] = filtered_emissions[c].apply(pd.to_numeric, errors='coerce')
    
filtered_emissions.index = filtered_emissions.index.map(lambda x: int(x.replace(' ','')[-4:]))
filtered_emissions /= 1000

filtered_emissions.shape

C:\Users\GEBRUI~1\AppData\Local\Temp/ipykernel_16700/1373398677.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_emissions[c] = filtered_emissions[c].apply(pd.to_numeric, errors='coerce')


(8, 489)

In [41]:
renamed_emissions = map_columns(filtered_emissions, 'Installatie')
grouped_emissions = renamed_emissions.groupby(renamed_emissions.columns, axis=1).sum()

race(grouped_emissions, 'installatie', pause=0)

In [42]:
# Now per company
renamed_emissions = map_columns(filtered_emissions, 'Bedrijf')
grouped_emissions = renamed_emissions.groupby(renamed_emissions.columns, axis=1).sum()



We make a few adjudstments where there are joint ventures

In [43]:
grouped_emissions['Total'] += round(grouped_emissions['Zeeland Refinery N.V.'] * 0.55)
grouped_emissions['LUKoil'] = round(grouped_emissions['Zeeland Refinery N.V.'] * 0.45)
grouped_emissions.drop('Zeeland Refinery N.V.', axis=1, inplace=True)


In [44]:
grouped_emissions['PZEM'] = round(grouped_emissions['Sloe Centrale B.V.'] * 0.50)
grouped_emissions['EDF'] = round(grouped_emissions['Sloe Centrale B.V.'] * 0.50)
grouped_emissions.drop('Sloe Centrale B.V.', axis=1, inplace=True)


In [45]:
race(grouped_emissions, 'bedrijf', pause=0)

For further analysis

In [46]:
total = grouped_emissions.sum(axis=0).sort_values(ascending=False)
annual = grouped_emissions.sum(axis=1)
last_year = grouped_emissions.iloc[-1,:].T.sort_values(ascending=False)

In [47]:
# Concat the two output files 
# make sure output file is not in directory otherwise doesnt work from notebook
# nullsrc is output size
# the two scales are input sizes
# the overlay is where number 2 should start

! ffmpeg -i installatie.mp4 -i bedrijf.mp4 -filter_complex " nullsrc=size=1664x504 [base];[0:v] setpts=PTS-STARTPTS, scale=864x504 [upperleft]; [1:v] setpts=PTS-STARTPTS, scale=864x504 [upperright]; [base][upperleft] overlay=shortest=1 [tmp1]; [tmp1][upperright] overlay=x=800" -c:v libx264 total4.mp4

ffmpeg version N-105398-g2f323b1978-20220129 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 11.2.0 (crosstool-NG 1.24.0.498_5075e1f)
  configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-w64-mingw32- --arch=x86_64 --target-os=mingw32 --enable-gpl --enable-version3 --disable-debug --disable-w32threads --enable-pthreads --enable-iconv --enable-libxml2 --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-lzma --enable-fontconfig --enable-libvorbis --enable-opencl --disable-libpulse --enable-libvmaf --disable-libxcb --disable-xlib --enable-amf --enable-libaom --enable-avisynth --enable-libdav1d --enable-libdavs2 --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --enable-frei0r --enable-libgme --enable-libass --enable-libbluray --enable-libmp3lame --enable-libopus --enable-librist --enable-libtheora --enable-libvpx --enable-libwebp --enable-lv2 --enable-libmfx --enable-libopencore-